In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
df = pd.read_csv("data/Hobart_airport_daily_precip.csv")

In [5]:
df.head

<bound method NDFrame.head of       Product code  Bureau of Meteorology station number  Year  Month  Day  \
0       IDCJAC0009                                 94008  1958      1    1   
1       IDCJAC0009                                 94008  1958      1    2   
2       IDCJAC0009                                 94008  1958      1    3   
3       IDCJAC0009                                 94008  1958      1    4   
4       IDCJAC0009                                 94008  1958      1    5   
5       IDCJAC0009                                 94008  1958      1    6   
6       IDCJAC0009                                 94008  1958      1    7   
7       IDCJAC0009                                 94008  1958      1    8   
8       IDCJAC0009                                 94008  1958      1    9   
9       IDCJAC0009                                 94008  1958      1   10   
10      IDCJAC0009                                 94008  1958      1   11   
11      IDCJAC0009                